In [1]:
import cv2,numpy as np,pandas as pd,os,cupy as cp
from sklearn.cluster import KMeans as KM

In [2]:
def does(x):
    pass

    
def coordi(items,iceri):
    max_x=0
    min_x=0
    max_y=0
    min_y=0
    for things in items:
        
        things=things[0]
        min_x=l=things[0]
        min_y=m=things[1]
        if (l>max_x):
            max_x=l
        if(l<min_x):
            min_x=l
        if (m>max_y):
            max_y=m
        if(m<min_y):
            min_y=m    
    return((min_x-iceri,min_y-iceri),(max_x+iceri,max_y+iceri))
def mapper(contors):
     dicti={}
     for it in contors:
               
        l,n,m=it.shape
        # if(l>10 and l<9000):
            
        cnt = it
        M = cv2.moments(cnt)
        if (M['m00'] != 0):
            cx = int(M['m10']/M['m00'])
            dicti[cx]=it
            
     return dicti    
def mk_box(image,kernel,iteration,ker_change,change_width,per_white ,y_jump):
    #this function basically makes the white lines over text
    cv2.imwrite("temp//unknown.png",image)
    y,x=image.shape
    h,w=kernel.shape
    black=np.zeros((y,x)).astype(np.uint8)
    for pqr in range (0,iteration):
        if (x>w and y>h):
            image=cv2.imread("temp//unknown.png")
            ones=np.ones((h,w)).astype(np.uint8)*255
            lim_upper=(y//h)
            lim_down=(x//w)
            for j in range (0,lim_upper):
                u=h*j +j*y_jump
                 
                for i in range (0,lim_down):
                    try:
                        l=w*i 
                        
                        check=image[u:u+h,l:l+w]
                        if (np.mean(check==255)>per_white):
                            black[u:u+h,l:l+w]=ones
                    except:
                        break
                 
                        
                         
                           
        else:
            print("incorrect dimensations from mk_box")
            break
        image=cv2.imwrite("temp//unknown.png",image)
        if (ker_change==True):
            w=w+change_width
        
            
         
    return black 
    
def mirror_symmetry(image):
    #should be use before mk_box and the image size should be even
    w,h=image.shape
    image_1=image[:,0:h//2]
    image_2=image[:,h//2:h]#even ya odd ho per hameha dono matrix ke size dame hogi agar x.5 ko x likhe to
    print(image.shape,image_1.shape,image_2.shape)
    common_same=cv2.bitwise_and(image_1,image_2)
    image_f=np.hstack((common_same,common_same))
     
    final=cv2.subtract(image,image_f)
    return final


def filter(img):
    """input should be an grey image"""
    # img_grey=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)    #clustered grey image
    img_grey=img
    bin1 = cv2.adaptiveThreshold(img_grey,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)#cv2.ADAPTIVE_THRESH_MEAN_C
    _,bin=cv2.threshold(img_grey,127,255,cv2.THRESH_BINARY)#cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    # cv2.imshow("bin_1",bin1)
    # cv2.imshow("bin_",bin)
    bin=cv2.bitwise_or(bin,bin1)
    bin=cv2.bitwise_not(bin)
    cv2.imshow("grey",img_grey)
    return bin

def clustering(img,no_of_clusters):
    img=img/255
    img_clus=np.reshape(img,(-1,3))
    km=KM(n_clusters=no_of_clusters).fit(img_clus)
    new_img=km.cluster_centers_[km.labels_]
    new_img=(new_img*255).astype(np.uint8)
    new_img=np.reshape(new_img,(im,si,3))
    return new_img 
def noise_under_nose(img,kernel,iteration):
    """this will take grey image"""
    y,x=img.shape
    h,w=kernel.shape
    image=img
    sq_img=np.array(image).astype(np.int16)
    sq_img=np.square(sq_img)
    black=np.zeros((y,x)).astype(np.int16)
    for pqr in range (0,iteration):
        if (x>w and y>h):
            lim_upper=(y//h)
            lim_down=(x//w)
            for j in range (0,lim_upper):
                u=h*j 
                for i in range (0,lim_down):
                     
                    l=w*i
                    check=image[u:u+h,l:l+w]#is is numpy
                    sq_chk=sq_img[u:u+h,l:l+w]#it is cupy
                    meanss=int(np.mean(check))
                    if (meanss==255 or meanss== 0):
                        continue
                    mean=np.mean(sq_chk)
                    sqrt_mean=np.sqrt(mean)
                    final=np.abs(check-sqrt_mean)
                    black[u:u+h,l:l+w]=final
                                  
        else:
            print("incorrect dimensations from nose")
            break
    return black.astype(np.uint8)                       

In [3]:
no_of_clusters=2
contor_wide=5 
brightness = 10 
contrast = 1

In [6]:
# x=no_of_clusters
i=0
blanl_loc="temp//blank.png"
loc="form_raw_data"
im,si=(800,800)
# cv2.namedWindow("clusters")
# cv2.createTrackbar("cluster","clusters",no_of_clusters,200,does)
for files in os.listdir(loc): 
    print(files)
    path=loc+"//"+files
    print(path)
    img=cv2.imread(path)
    pl,pm,_=img.shape
                                                    # pl=(pl//2)*2#these was checked for the use of symmetricity,since that function is not in use so leaved
                                                    # pm=(pm//2)*2 
                                                    # img=cv2.resize(img,(pl,pm))
    imgs=img.copy()
    im,si,_=imgs.shape
    blank=(np.ones((im,si,3))*255).astype(np.uint8) 
    cv2.imwrite(blanl_loc,blank)
    
   #we have only load the files yet  
    
    img = cv2.addWeighted(img, contrast, np.zeros(img.shape, img.dtype), 0, brightness)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv) 
    s = np.clip(s * 3, 0, 255).astype(np.uint8)  # 1.5x satura
    # Merge and convert back
    vibrant_hsv = cv2.merge([h, s, v])
    vibrant_img = cv2.cvtColor(vibrant_hsv, cv2.COLOR_HSV2BGR)
   
    denoised_image = cv2.GaussianBlur(vibrant_img, (3, 3), 0)
    new_img=denoised_image
     
    #filtered_the_image
#   or
    #pre_filter

    img_grey=cv2.cvtColor(new_img,cv2.COLOR_RGB2GRAY)
    
    okey=noise_under_nose(img=img_grey,kernel=np.ones((3,3)),iteration=1)#make word recorganizable
    _,okey=cv2.threshold(okey,10,255,cv2.THRESH_BINARY)#assuming that atleast 10 x out of 255 color difference should be there for normal visiblity 

    
    pre_filter=mk_box(image=okey,kernel=np.ones((3,4)),iteration=4,ker_change=True,change_width=5,per_white=0.30,y_jump=0)
    let_see=mk_box(image=pre_filter,kernel=np.ones((30,pl//10)),iteration=12,ker_change=True,change_width=10,per_white=0.3,y_jump=1) 
    final_touch=let_see

                                                # bin=cv2.bitwise_and(img_grey,pre_filter)
                                                # bin=filter(bin)#imput is 3d image                      note the function is not in use
                                                # cv2.imshow("transfer",bin)
                                                #filtered binary image;#removed the noise from the image
    
    cv2.imshow("pre_filter",pre_filter)
    cv2.imshow("noise_nose",okey)
    cv2.imshow("let_see",let_see)

    #making contors from now
    
    c,h=cv2.findContours(final_touch,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)#phale bin tha
    counter_img=cv2.drawContours(imgs,c,-1,(0,0,255),2) 
    cv2.imshow("counter_img",counter_img)
    # dicti=mapper(c) -> since now we are making boxex so no need to sort
    # keys=sorted(dicti)
    i=i+1
    for it in c:
         
        fil_nam=f"temp//data_{i}.png"
        cv2.imwrite(fil_nam,imgs)
        blank=cv2.imread(fil_nam)
        p,y=coordi(it,contor_wide)
        y0,y1=p[1],y[1]
        x0,x1=p[0],y[0]
        blank= cv2.rectangle(blank,p,y,(0,255,0),2,1) 
        cv2.imwrite(fil_nam,blank)
        cv2.waitKey(1)   
    cv2.imshow("finalaa",blank)
    cv2.imwrite(fil_nam,blank)
    if cv2.waitKey(1) &0xff==ord("q"):
        break
    cv2.waitKey(1)
    
cv2.destroyAllWindows()

a1.jpg
form_raw_data//a1.jpg


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7672\146778353.py:134: RuntimeWarning: invalid value encountered in sqrt
  sqrt_mean=np.sqrt(mean)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7672\146778353.py:136: RuntimeWarning: invalid value encountered in cast
  black[u:u+h,l:l+w]=final


a1.png
form_raw_data//a1.png
a10.png
form_raw_data//a10.png
a11.jpg
form_raw_data//a11.jpg
a2.png
form_raw_data//a2.png
a3.png
form_raw_data//a3.png
a4.png
form_raw_data//a4.png
a5.png
form_raw_data//a5.png
a6.png
form_raw_data//a6.png


In [5]:
x=np.ones((100,100)).astype(np.uint8)*120
y=cp.array(x).astype(np.int16)
y=cp.square(y)
y=y.get()
print(x,y)

[[120 120 120 ... 120 120 120]
 [120 120 120 ... 120 120 120]
 [120 120 120 ... 120 120 120]
 ...
 [120 120 120 ... 120 120 120]
 [120 120 120 ... 120 120 120]
 [120 120 120 ... 120 120 120]] [[14400 14400 14400 ... 14400 14400 14400]
 [14400 14400 14400 ... 14400 14400 14400]
 [14400 14400 14400 ... 14400 14400 14400]
 ...
 [14400 14400 14400 ... 14400 14400 14400]
 [14400 14400 14400 ... 14400 14400 14400]
 [14400 14400 14400 ... 14400 14400 14400]]
